#### Representatives extraction from bgsu database (3A resolution constraint)
[Link](http://rna.bgsu.edu/rna3dhub/nrlist/release/3.242/3.0A)

In [1]:
import os
PATH = "../external-files/nrlist_3.242_3.0A.csv"
with open(PATH) as f:
    lines = f.readlines()
print(len(lines))

1646


In [2]:
pdbs = [l.strip() for l in lines]
pdbs = [p.split(',')[1:2] for p in pdbs]

def additional_chains(pdb):
    if '+' in pdb:
        pdb_ab = pdb.split("+")
        pdb_ab = [p.split("|") for p in pdb_ab]
        core = "_".join(pdb_ab[0][:-1])
        pdb = f"{core}_{pdb_ab[0][-1]}-{pdb_ab[1][-1]}"
    else:
        pdb = pdb.split("|")
        pdb = "_".join(pdb)
    return pdb

def combine_ids(pdb_line):
    pdbs = [p.replace("\"", "").replace("\'", "") for p in pdb_line]
    pdbs = [additional_chains(p) for p in pdbs]
    return pdbs

pdbs = [combine_ids(p) for p in pdbs]
pdbs = [p[0] for p in pdbs]
pdb_repres = [f'{p.split("_")[0]}_{p.split("_")[2]}' for p in pdbs]
print(pdbs[:10])

with open('../metadata/representants.txt', 'w') as f:
    out = "\n".join(pdbs)
    f.writelines(out)

['6CFJ_1_1x', '5L4O_1_A', '5E81_1_2K', '6I0V_1_B', '6YDP_1_BB', '2DU3_1_D', '1KUQ_1_B', '5K77_1_X', '5J7L_1_DB', '3BT7_1_D']


In [3]:
import os
train_pdbs = "../external-files/PDB_dataset/TR1_sequences/"
test_pdbs1 = "../external-files/PDB_dataset/TS1_sequences/"
test_pdbs2 = "../external-files/PDB_dataset/TS2_sequences/"

def get_pdb_ids(path):
    pdbs = os.listdir(path)
    pdbs = [p.replace("-", "_") for p in pdbs]
    pdbs = [f'{p[:4].upper()}{p[4:]}' for p in pdbs]
    return pdbs

trpdbs = get_pdb_ids(train_pdbs)
tspdbs1 = get_pdb_ids(test_pdbs1)
tspdbs2 = get_pdb_ids(test_pdbs2)
tspdbs1.extend(tspdbs2)
print(trpdbs[:10])
print(len(trpdbs), len(tspdbs1), len(tspdbs2))

['3SKI_1_A', '2HVY_1_E', '4PMI_1_A', '1FFK_1_9', '2CSX_1_C', '2ZNI_1_C', '4Y1J_1_A', '4KZD_1_R', '4V83_1_AV', '2A64_1_A']
120 106 39


Cząsteczki ze zbioru treningowego SPOT-RNA wśród reprezentantów:

In [5]:
intersection = set(pdbs).intersection(trpdbs)
diffrerence = set(pdbs).difference(trpdbs)
print(f'Intersection: {len(intersection)}')
print(f'Diffrefence: {len(diffrerence)}')

Intersection: 57
Diffrefence: 1589
